In [2]:
import numpy as np
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from scipy import sparse
import plotly.graph_objects as go


In [5]:
N_points = 100
up_bound = 1
dn_bound = -1

del_x = (up_bound - dn_bound)/(N_points - 1)

x_grid,y_grid = np.meshgrid(
    np.linspace(dn_bound,up_bound,N_points,dtype=float),
    np.linspace(dn_bound,up_bound,N_points,dtype=float)
    )

D = sparse.spdiags(
    np.array([np.ones(N_points), -2*np.ones(N_points), np.ones(N_points)]),
    np.array([-1, 0,1]),
    N_points, N_points)
P_2 = sparse.kronsum(D,D)
T_hat = -1/2 * P_2

def potential (x_grid,y_grid):
    return ((x_grid)**2 + (y_grid)**2)/2

V_hat = sparse.diags(potential(x_grid,y_grid).reshape(N_points*N_points),(0))

H_hat = T_hat + V_hat

E , psi = eigsh(H_hat,which='SM')



In [24]:
for i in range(6):
    fig = go.Figure(go.Surface(x=x_grid,y=y_grid,z=psi.T[i].reshape((N_points,N_points)),
        contours = {
            "z": {"show": True, "start": -1, "end": 1, "size": 0.005}
        },    
        colorscale='Aggrnyl',
        colorbar=dict(thickness=20, ticklen=4),
    ))
    fig.update_layout(title = "No.{i} Wavefunction ",template='plotly_dark',showlegend=True)
    fig.write_html(f"\\result\\2-d_harmonic_oscillator_potential\\py-no{i}state.html", include_plotlyjs='cdn')


In [ ]:
[]